In [25]:
import sklearn
import pandas as pd
import numpy as np
import re

In [26]:
F17_df = pd.read_csv('./Dataset/FIFA17_official_data.csv')
F18_df = pd.read_csv('./Dataset/FIFA18_official_data.csv')
F19_df = pd.read_csv('./Dataset/FIFA19_official_data.csv')
F20_df = pd.read_csv('./Dataset/FIFA20_official_data.csv')
F21_df = pd.read_csv('./Dataset/FIFA21_official_data.csv')
F22_df = pd.read_csv('./Dataset/FIFA22_official_data.csv')
datalist = [F17_df, F18_df, F19_df, F20_df, F21_df, F22_df]

In [27]:
# print(set(F18_df.columns) - set(F17_df.columns))
# print(set(F19_df.columns) - set(F18_df.columns))
# print(set(F20_df.columns) - set(F19_df.columns))
# print(set(F21_df.columns) - set(F20_df.columns))
# print(set(F22_df.columns) - set(F21_df.columns))
curr_set = F17_df
for next_set in datalist[1:]:
    print(set(next_set) - set(curr_set))
    curr_set = next_set

for dataset in datalist:
    if 'Release Clause' in list(dataset.columns):
        dataset.drop("Release Clause", axis = 1, inplace = True)
    if 'DefensiveAwareness' in list(dataset.columns):
        dataset.drop("DefensiveAwareness", axis = 1, inplace = True)
print("--------------------")
curr_set = F17_df
for next_set in datalist[1:]:
    print(set(next_set) - set(curr_set))
    curr_set = next_set

{'Release Clause'}
set()
{'DefensiveAwareness'}
set()
set()
--------------------
set()
set()
set()
set()
set()


In [28]:
#Now we can join the datasets\
F_all_df = pd.concat(datalist, ignore_index=True)
F_all_df.columns

Index(['ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag', 'Overall',
       'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'Crossing', 'Finishing', 'HeadingAccuracy',
       'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy',
       'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility',
       'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength',
       'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle',
       'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes',
       'Best Position', 'Best Overall Rating'],
      dtype='object')

In [29]:
Unimportant_features  = ["Photo", "Flag", "Club Logo", "Wage", "Special", "International Reputation", "Work Rate", "Body Type", "Real Face", "Jersey Number", "Contract Valid Until", "Best Overall Rating"]
F_all_df.drop(Unimportant_features, axis = 1, inplace = True)
F_all_df["Ovr_pot"] = (F_all_df["Overall"] + F_all_df["Potential"])/2

In [30]:
# One-hot encode Preferred_foot
F_all_df = pd.get_dummies(F_all_df, columns = ["Preferred Foot"])
F_all_df

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Weak Foot,Skill Moves,...,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Ovr_pot,Preferred Foot_Left,Preferred Foot_Right
0,176580,L. Suárez,29,Uruguay,92,92,FC Barcelona,€83M,4.0,4.0,...,38.0,27.0,25.0,31.0,33.0,37.0,ST,92.0,0,1
1,178518,R. Nainggolan,28,Belgium,86,86,Roma,€37.5M,3.0,3.0,...,88.0,11.0,11.0,14.0,8.0,11.0,CDM,86.0,0,1
2,181872,A. Vidal,29,Chile,87,87,FC Bayern München,€41.5M,4.0,3.0,...,84.0,4.0,2.0,4.0,2.0,4.0,CDM,87.0,0,1
3,197445,D. Alaba,24,Austria,86,89,FC Bayern München,€41.5M,4.0,3.0,...,83.0,5.0,7.0,14.0,15.0,9.0,LB,87.5,1,0
4,195864,P. Pogba,23,France,88,94,Manchester United,€71.5M,4.0,5.0,...,73.0,5.0,6.0,2.0,4.0,3.0,CAM,91.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104347,240558,18 L. Clayton,17,England,53,70,Cheltenham Town,€100K,2.0,1.0,...,12.0,55.0,54.0,52.0,50.0,59.0,GK,61.5,0,1
104348,262846,�. Dobre,20,Romania,53,63,FC Academica Clinceni,€180K,2.0,1.0,...,12.0,57.0,52.0,53.0,48.0,58.0,GK,58.0,0,1
104349,241317,21 Xue Qinghao,19,China PR,47,60,Shanghai Shenhua FC,€100K,2.0,1.0,...,9.0,49.0,48.0,45.0,38.0,52.0,GK,53.5,0,1
104350,259646,A. Shaikh,18,India,47,67,ATK Mohun Bagan FC,€110K,3.0,1.0,...,13.0,49.0,41.0,39.0,45.0,49.0,GK,57.0,0,1


In [31]:
detailed_features = ['Age','Overall', 'Potential', 'Weak Foot', 'Skill Moves', 'Height', 'Weight', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes', "Ovr_pot", 'Preferred Foot_Left', 'Preferred Foot_Right']

In [32]:
F_all_df[detailed_features] = F_all_df[detailed_features].fillna(0)
F_all_df

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Weak Foot,Skill Moves,...,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Ovr_pot,Preferred Foot_Left,Preferred Foot_Right
0,176580,L. Suárez,29,Uruguay,92,92,FC Barcelona,€83M,4.0,4.0,...,38.0,27.0,25.0,31.0,33.0,37.0,ST,92.0,0,1
1,178518,R. Nainggolan,28,Belgium,86,86,Roma,€37.5M,3.0,3.0,...,88.0,11.0,11.0,14.0,8.0,11.0,CDM,86.0,0,1
2,181872,A. Vidal,29,Chile,87,87,FC Bayern München,€41.5M,4.0,3.0,...,84.0,4.0,2.0,4.0,2.0,4.0,CDM,87.0,0,1
3,197445,D. Alaba,24,Austria,86,89,FC Bayern München,€41.5M,4.0,3.0,...,83.0,5.0,7.0,14.0,15.0,9.0,LB,87.5,1,0
4,195864,P. Pogba,23,France,88,94,Manchester United,€71.5M,4.0,5.0,...,73.0,5.0,6.0,2.0,4.0,3.0,CAM,91.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104347,240558,18 L. Clayton,17,England,53,70,Cheltenham Town,€100K,2.0,1.0,...,12.0,55.0,54.0,52.0,50.0,59.0,GK,61.5,0,1
104348,262846,�. Dobre,20,Romania,53,63,FC Academica Clinceni,€180K,2.0,1.0,...,12.0,57.0,52.0,53.0,48.0,58.0,GK,58.0,0,1
104349,241317,21 Xue Qinghao,19,China PR,47,60,Shanghai Shenhua FC,€100K,2.0,1.0,...,9.0,49.0,48.0,45.0,38.0,52.0,GK,53.5,0,1
104350,259646,A. Shaikh,18,India,47,67,ATK Mohun Bagan FC,€110K,3.0,1.0,...,13.0,49.0,41.0,39.0,45.0,49.0,GK,57.0,0,1


In [33]:
# Clean height column
feet_inches_re = re.compile(r"(\d)'(\d+)")

# Define a function to convert feet'inches to cm
def feet_inches_to_cm(s):
    match = feet_inches_re.match(s)
    if match:
        feet = int(match.group(1))
        inches = int(match.group(2))
        return round((feet*12 + inches) * 2.54, 2)
    else:
        return float(s.replace('cm', ''))
F_all_df['Height'] = pd.to_numeric(F_all_df['Height'].apply(feet_inches_to_cm))

In [34]:
F_all_df['Height'].value_counts()

182.88    13593
177.80    12221
175.26    10661
180.34    10380
187.96     9448
          ...  
159.00        3
156.00        2
155.00        1
157.00        1
206.00        1
Name: Height, Length: 71, dtype: int64

In [35]:
F_all_df['Weight'].value_counts()

154lbs    7042
165lbs    6815
176lbs    4958
159lbs    4800
172lbs    4717
          ... 
104kg        1
51kg         1
53kg         1
106lbs       1
105kg        1
Name: Weight, Length: 117, dtype: int64

In [36]:
# Clean Weight Column
def convert_weight(weight_str):
    if 'kg' in weight_str:
        # If weight is in kg, remove the 'kg' suffix and return as float
        return float(weight_str.replace('kg', ''))
    elif 'lbs' in weight_str:
        # If weight is in lbs, convert to kg and return as float
        return float(weight_str.replace('lbs', '')) * 0.453592
    else:
        print("oops")
        # Return NaN if weight format is not recognized
        return pd.np.nan

# Apply the function to the Weight column and create a new column 'Weight_kg'
F_all_df['Weight'] = F_all_df['Weight'].apply(convert_weight)

In [37]:
F_all_df['Weight'].value_counts()

69.853168     7042
74.842680     6815
79.832192     4958
72.121128     4800
78.017824     4717
              ... 
104.000000       1
51.000000        1
53.000000        1
48.080752        1
105.000000       1
Name: Weight, Length: 117, dtype: int64

In [39]:
scaler = sklearn.preprocessing.StandardScaler()
F_norm_df = F_all_df.copy(deep = True)
F_norm_df[detailed_features] = scaler.fit_transform(F_norm_df[detailed_features].to_numpy())
F_norm_df

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Weak Foot,Skill Moves,...,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Ovr_pot,Preferred Foot_Left,Preferred Foot_Right
0,176580,L. Suárez,0.873366,Uruguay,3.647373,3.353318,FC Barcelona,€83M,1.552616,2.090192,...,-0.366832,0.618657,0.538418,0.922738,1.007493,1.168766,ST,3.876224,-0.558918,0.558918
1,178518,R. Nainggolan,0.665442,Belgium,2.786419,2.355094,Roma,€37.5M,0.053943,0.792811,...,1.978031,-0.301205,-0.303018,-0.118727,-0.479603,-0.302032,CDM,2.855590,-0.558918,0.558918
2,181872,A. Vidal,0.873366,Chile,2.929911,2.521465,FC Bayern München,€41.5M,1.552616,0.792811,...,1.790442,-0.703645,-0.843941,-0.731353,-0.836506,-0.698015,CDM,3.025696,-0.558918,0.558918
3,197445,D. Alaba,-0.166257,Austria,2.786419,2.854206,FC Bayern München,€41.5M,1.552616,0.792811,...,1.743544,-0.646154,-0.543428,-0.118727,-0.063216,-0.415170,LB,3.110749,1.789172,-1.789172
4,195864,P. Pogba,-0.374181,France,3.073404,3.686059,Manchester United,€71.5M,1.552616,3.387573,...,1.274572,-0.646154,-0.603530,-0.853879,-0.717538,-0.754585,CAM,3.706119,-0.558918,0.558918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104347,240558,18 L. Clayton,-1.621729,England,-1.948825,-0.306835,Cheltenham Town,€100K,-1.444731,-1.801951,...,-1.586161,2.228417,2.281393,2.209254,2.018719,2.413286,GK,-1.311998,-0.558918,0.558918
104348,262846,�. Dobre,-0.997955,Romania,-1.948825,-1.471429,FC Academica Clinceni,€180K,-1.444731,-1.801951,...,-1.586161,2.343399,2.161187,2.270516,1.899751,2.356717,GK,-1.907368,-0.558918,0.558918
104349,241317,21 Xue Qinghao,-1.205880,China PR,-2.809779,-1.970541,Shanghai Shenhua FC,€100K,-1.444731,-1.801951,...,-1.726852,1.883468,1.920777,1.780415,1.304912,2.017302,GK,-2.672843,-0.558918,0.558918
104350,259646,A. Shaikh,-1.413805,India,-2.809779,-0.805947,ATK Mohun Bagan FC,€110K,0.053943,-1.801951,...,-1.539263,1.883468,1.500059,1.412839,1.721299,1.847595,GK,-2.077473,-0.558918,0.558918


In [44]:
striker_df = F_norm_df[(F_norm_df["Best Position"] == "ST") | (F_norm_df["Best Position"] == "CF") | (F_norm_df["Best Position"] == "LF")]
striker_df  =  striker_df.reset_index(drop = True)
striker_df

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Weak Foot,Skill Moves,...,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Ovr_pot,Preferred Foot_Left,Preferred Foot_Right
0,176580,L. Suárez,0.873366,Uruguay,3.647373,3.353318,FC Barcelona,€83M,1.552616,2.090192,...,-0.366832,0.618657,0.538418,0.922738,1.007493,1.168766,ST,3.876224,-0.558918,0.558918
1,173731,G. Bale,0.249592,Wales,3.360388,3.020577,Real Madrid,€72M,0.053943,2.090192,...,0.289730,-0.071239,-0.062607,-0.302515,-0.658054,-0.584877,ST,3.536013,1.789172,-1.789172
2,20801,Cristiano Ronaldo,1.289216,Portugal,3.934357,3.686059,Real Madrid,€87M,1.552616,3.387573,...,-1.070291,-0.531171,-0.303018,-0.057464,-0.122700,-0.302032,ST,4.216436,-0.558918,0.558918
3,184941,A. Sánchez,0.457517,Chile,3.073404,2.687836,Arsenal,€55M,0.053943,2.090192,...,-0.507524,-0.358697,-0.363120,-0.057464,-0.241667,-0.188893,ST,3.195802,-0.558918,0.558918
4,54050,W. Rooney,1.081291,England,2.212450,1.689612,Manchester United,€18.5M,1.552616,0.792811,...,-0.413729,-0.358697,-0.303018,-0.179990,-0.479603,-0.528308,ST,2.175168,-0.558918,0.558918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15809,262034,R. Mahon,-1.413805,Republic of Ireland,-2.666287,-1.970541,Bohemian FC,€110K,0.053943,-0.504570,...,-1.070291,-0.531171,-0.122710,-0.608828,-0.479603,-0.075755,ST,-2.587790,1.789172,-1.789172
15810,259650,F. Molla,-1.205880,India,-2.809779,-0.805947,ATK Mohun Bagan FC,€130K,-1.444731,-0.504570,...,-1.633058,-0.473679,-0.603530,-0.118727,-0.301151,-0.132324,ST,-2.077473,-0.558918,0.558918
15811,161659,07 Danilo,-0.166257,Brazil,-0.370411,-0.805947,Gallos Blancos de Querétaro,€0,0.053943,-1.801951,...,-2.148928,-0.301205,-0.603530,2.515567,-0.063216,-0.245462,ST,-0.631575,1.789172,-1.789172
15812,150597,09 J. Moukoko,-0.374181,Congo DR,-1.805333,-1.637800,NaN,€0,0.053943,-1.801951,...,-2.148928,-0.473679,0.538418,1.474102,0.531623,0.489936,ST,-1.907368,-0.558918,0.558918


In [45]:
striker_df.columns

Index(['ID', 'Name', 'Age', 'Nationality', 'Overall', 'Potential', 'Club',
       'Value', 'Weak Foot', 'Skill Moves', 'Position', 'Joined',
       'Loaned From', 'Height', 'Weight', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes', 'Best Position', 'Ovr_pot',
       'Preferred Foot_Left', 'Preferred Foot_Right'],
      dtype='object')

In [46]:
ST_detailed_features = ['Weak Foot', 'Skill Moves', 'Height', 'Weight', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle']
# Left and Right foot not used as this is central position.

In [71]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
X = striker_df[ST_detailed_features].to_numpy()
y = striker_df["Ovr_pot"].to_numpy()
# define feature selection
fs = SelectKBest(score_func=f_regression, k = 10)
# apply feature selection
X_selected = fs.fit_transform(X, y)

In [72]:
ST_essential_ft = list(fs.get_feature_names_out(ST_detailed_features))
ST_essential_ft

['Finishing',
 'HeadingAccuracy',
 'ShortPassing',
 'Volleys',
 'Dribbling',
 'BallControl',
 'Reactions',
 'ShotPower',
 'LongShots',
 'Positioning']